In [1]:
import os
import re
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm # Progress Bar
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import transformers
from transformers import DistilBertTokenizer, TFDistilBertModel, DistilBertConfig
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import warnings
from transformers import logging as hf_logging
hf_logging.set_verbosity_error() # Hidding Huggingface Warnings
warnings.filterwarnings("ignore")

2022-12-10 21:29:13.114897: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-10 21:29:13.425341: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-10 21:29:13.425373: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-10 21:29:14.719317: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder

In [18]:
from ipynb.fs.full.preprocess_dataset import generate_dataframe

In [19]:
df = generate_dataframe(10)

df = df.rename(columns={'Post': 'processed'})

df

,processed,Label,code
0,viable option youll leaving wife behind youd p...,Supportive,4
1,hard appreciate notion could meet someone else...,Ideation,2
2,hi last night wa sitting ledge window contempl...,Behavior,1
3,tried kill self failed badly cause moment want...,Attempt,0
4,hi nem3030 sort thing enjoy personally always ...,Ideation,2
...,...,...,...
58,Non-pyogenic bacterial infection of skin Fuso...,Indicator,3
133,Nodular basal cell carcinoma of skin Basal ce...,Indicator,3
2,Communication disorder Depressive adjustment ...,Indicator,3
20,Family history of central nervous system diso...,Indicator,3


In [21]:
df.reset_index(inplace=True)

In [22]:
df=df.drop(columns=['index'])

In [23]:
df.dropna()

,level_0,processed,Label,code
0,0,viable option youll leaving wife behind youd p...,Supportive,4
1,1,hard appreciate notion could meet someone else...,Ideation,2
2,2,hi last night wa sitting ledge window contempl...,Behavior,1
3,3,tried kill self failed badly cause moment want...,Attempt,0
4,4,hi nem3030 sort thing enjoy personally always ...,Ideation,2
...,...,...,...,...
725,725,Non-pyogenic bacterial infection of skin Fuso...,Indicator,3
726,726,Nodular basal cell carcinoma of skin Basal ce...,Indicator,3
727,727,Communication disorder Depressive adjustment ...,Indicator,3
728,728,Family history of central nervous system diso...,Indicator,3


In [24]:
df

,level_0,processed,Label,code
0,0,viable option youll leaving wife behind youd p...,Supportive,4
1,1,hard appreciate notion could meet someone else...,Ideation,2
2,2,hi last night wa sitting ledge window contempl...,Behavior,1
3,3,tried kill self failed badly cause moment want...,Attempt,0
4,4,hi nem3030 sort thing enjoy personally always ...,Ideation,2
...,...,...,...,...
725,725,Non-pyogenic bacterial infection of skin Fuso...,Indicator,3
726,726,Nodular basal cell carcinoma of skin Basal ce...,Indicator,3
727,727,Communication disorder Depressive adjustment ...,Indicator,3
728,728,Family history of central nervous system diso...,Indicator,3


In [11]:
# Get the number training samples
num_train_samples = int(df.shape[0] * 0.9)

x_train = df['processed'][:num_train_samples]
y_train = df['code'][:num_train_samples]
x_val = df['processed'][num_train_samples:]
y_val = df['code'][num_train_samples:]

### Sanity Check
assert(len(x_train) + len(x_val) == df.shape[0])

print(f"{len(x_train)} training examples\n{len(x_val)} validation examples")

test_df = df.tail(30)
x_test = test_df['processed']
y_test = test_df['code']
print(f"{len(x_test)} testing examples")

657 training examples
73 validation examples
30 testing examples


In [2]:
import pandas as  pd

In [3]:
df=pd.read_csv('/home/ifte-home/Documents/mental_health/suicide/CSSRS/500_Reddit_users_posts_labels.csv')

In [46]:
df

,User,Post,Label
0,user-0,"['Its not a viable option, and youll be leavin...",Supportive
1,user-1,['It can be hard to appreciate the notion that...,Ideation
2,user-2,"['Hi, so last night i was sitting on the ledge...",Behavior
3,user-3,['I tried to kill my self once and failed badl...,Attempt
4,user-4,['Hi NEM3030. What sorts of things do you enjo...,Ideation
...,...,...,...
495,user-495,"['Its not the end, it just feels that way. Or ...",Supportive
496,user-496,"['It was a skype call, but she ended it and Ve...",Indicator
497,user-497,['That sounds really weird.Maybe you were Dist...,Supportive
498,user-498,['Dont know there as dumb as it sounds I feel ...,Attempt


In [11]:
from ipynb.fs.full.preprocess_dataset import generate_dataframe

In [8]:
seq_len = a

[64,
 13,
 14,
 6,
 218,
 140,
 110,
 4,
 89,
 107,
 13,
 45,
 19,
 61,
 3,
 12,
 88,
 11,
 7,
 33,
 72,
 16,
 795,
 253,
 96,
 20,
 60,
 58,
 25,
 25,
 9,
 61,
 6,
 87,
 7,
 9,
 78,
 76,
 131,
 15,
 28,
 28,
 121,
 15,
 50,
 23,
 29,
 35,
 18,
 11,
 107,
 55,
 13,
 5,
 46,
 20,
 5,
 35,
 3,
 65,
 8,
 82,
 53,
 45,
 10,
 23,
 55,
 51,
 23,
 88,
 26,
 441,
 91,
 243,
 10,
 33,
 15,
 20,
 309,
 136,
 8,
 40,
 16,
 154,
 9,
 12,
 9,
 233,
 558,
 92,
 56,
 25,
 13,
 19,
 94,
 17,
 131,
 14,
 16,
 496,
 18,
 9,
 28,
 24,
 66,
 8,
 53,
 12,
 20,
 12,
 21,
 11,
 5,
 14,
 8,
 403,
 37,
 102,
 115,
 398,
 24,
 469,
 24,
 25,
 149,
 15,
 6,
 39,
 119,
 19,
 17,
 81,
 10,
 67,
 658,
 3,
 20,
 20,
 24,
 3,
 56,
 54,
 103,
 107,
 6,
 16,
 44,
 5,
 33,
 81,
 10,
 12,
 298,
 14,
 22,
 17,
 7,
 396,
 9,
 5,
 14,
 8,
 23,
 32,
 83,
 10,
 18,
 24,
 2,
 47,
 12,
 103,
 18,
 26,
 17,
 51,
 10,
 24,
 3,
 256,
 13,
 128,
 7,
 98,
 14,
 82,
 8,
 58,
 7,
 61,
 26,
 296,
 33,
 183,
 88,
 13,
 71,
 93,
 82,
 22

In [14]:
from matplotlib import colors

In [15]:
ax = sns.histplot(data=seq_len, log_scale=True, bins=20)
ax.set(ylabel='Sample Count', xlabel='Number of Tokens', title='Documents length Distribution in Twitter Dataset')
plt.show()

NameError: name 'sns' is not defined

In [48]:
df.Label.value_counts()

Ideation      171
Supportive    108
Indicator      99
Behavior       77
Attempt        45
Name: Label, dtype: int64

In [16]:
import numpy as np
import matplotlib.pyplot as plt

plt.plot(seq_len) # plotting by columns
plt.ylabel('Number of Tokens')
plt.xlabel('Number of Documents')
plt.title('Twitter dataset (Depression/Suicide) samples')
plt.show()

NameError: name 'seq_len' is not defined

In [25]:
# get length of all the messages in the train set
seq_len = [len(str(i).split()) for i in df['Processed'].values]
pd.Series(seq_len).hist(bins = 5)

KeyError: 'Processed'

In [12]:
model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(x_train, y_train)

### Make predictions
y_hat = model.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_hat))

Accuracy: 0.7333333333333333


In [14]:
MODEL_NAME='distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME) # Loading the tokenizer


In [15]:
max_val = 0
for sent in (x_train.tolist() + x_train.tolist() + x_val.tolist()):
    try:
        sent_tok_len = len(tokenizer.tokenize(sent))
        max_val = sent_tok_len if (sent_tok_len > max_val) else max_val
    except:
        pass
    
print(f"The maximum amount of tokens in the dataset is {max_val}")


The maximum amount of tokens in the dataset is 3759


In [59]:
MAX_LENGTH = max_val + 2

tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME,  
                                                add_special_tokens=True,
                                                max_length=MAX_LENGTH, 
                                                pad_to_max_length=True)

def tokenize(sentences, tokenizer):
    input_ids, input_masks, input_segments = [], [], []
    for sentence in tqdm(sentences):
        inputs = tokenizer.encode_plus(sentence, 
                                       add_special_tokens=True, 
                                       max_length=MAX_LENGTH, 
                                       pad_to_max_length=True, 
                                       return_attention_mask=True, 
                                       return_token_type_ids=True, 
                                       truncation=True)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
        input_segments.append(inputs['token_type_ids'])        
        
    return np.asarray(input_ids, dtype='int32'), np.asarray(input_masks, dtype='int32')

In [60]:
X_train = tokenize(x_train, tokenizer)
X_test = tokenize(x_test, tokenizer)
X_val = tokenize(x_val, tokenizer)

100%|██████████████████████████████████████████| 73/73 [00:00<00:00, 813.18it/s]


In [61]:
index_num = 5

print(df.loc[index_num]['code'], '\n')

print(x_train[index_num], '\n')

temp_tokens = tokenizer.tokenize(x_train[index_num])
temp_ids = tokenizer.encode(x_train[index_num])

print('ID\'s', 'Input Tokens', sep='\t')
for i in range(len(temp_ids)):
    if i == 0:
        print(temp_ids[i], '[CLS]', sep='\t')
        continue
    if i == len(temp_ids)-1:
        print(temp_ids[i], '[SEP]', sep='\t')
        break
    print(temp_ids[i], temp_tokens[i-1], sep='\t')

4 

since dont know dbt would mind explaining little bit really sorry hear someone supposed helping actually really mean thats definitely right fair people person talk im honestly sure shunned sound like dont know either im sure would anything tomorrow plan sound really pain make sure understand 30 ish cut arm 60 total im tired concerned safety throwaway_lives symptom wanting wake life bad dream sound like tired scared hitting rock bottom also pleasant thing experience dbt lady one conduct therapy session sound like feeling really scared much want kill im concerned safety especially since said youve carrying plan suicide chance could talk yeah bet would make really tired 9 day row lot day work hear youre feeling lot better ever need support always message like said schedule really sporadic get time shared thought feeling youve sharing therapist care coordinator even though care coordinator ha mean would hope would willing help im super concerned safety right youve said youve reopened w

In [63]:
config = DistilBertConfig.from_pretrained(MODEL_NAME, output_hidden_states=True, 
                                          output_attentions=True)
DistilBERT = TFDistilBertModel.from_pretrained(MODEL_NAME, config=config)

input_ids_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='input_token', dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='masked_token', dtype='int32') 

embedding_layer = DistilBERT(input_ids = input_ids_in, attention_mask = input_masks_in)[0]
X = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(embedding_layer)
X = tf.keras.layers.GlobalMaxPool1D()(X)
X = tf.keras.layers.Dense(64, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(5, activation='softmax')(X)

model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = X)

for layer in model.layers[:3]:
    layer.trainable = False

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 3761)]       0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 3761)]       0           []                               
                                                                                                  
 tf_distil_bert_model_1 (TFDist  TFBaseModelOutput(l  66362880   ['input_token[0][0]',            
 ilBertModel)                   ast_hidden_state=(N               'masked_token[0][0]']           
                                one, 3761, 768),                                                  
                                 hidden_states=((No                                           

In [65]:
### Create an output directory
output_dir = './model1_outputs'
if not os.path.exists(output_dir): ### If the file directory doesn't already exists,
    os.makedirs(output_dir) ### Make it please


In [66]:
model_checkpoint = ModelCheckpoint(filepath=output_dir+'/weights.{epoch:02d}.hdf5',
                                  save_weights_only=True)

early_stopping = EarlyStopping(patience=3, # Stop after 3 epochs of no improvement
                               monitor='val_loss', # Look at validation_loss
                               min_delta=0, # After 0 change
                               mode='min', # Stop when quantity has stopped decreasing
                               restore_best_weights=False, # Don't Restore the best weights
                               verbose=1) 

reduce_lr = ReduceLROnPlateau(monitor='val_loss', # Look at validation loss
                              min_lr=0.000001, # Lower bound of learning rate
                              patience=1, # Reduce after 1 with little change
                              mode='min', # Stop when quantity has stopped decreasing
                              factor=0.1, # Reduce by a factor of 1/10
                              min_delta=0.01, # Minimumn change needed
                              verbose=1)

In [67]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, 
                    y_train, 
                    epochs = 10,
                    batch_size=16, 
                    validation_data=(X_val, y_val), 
                    callbacks=[model_checkpoint, early_stopping, reduce_lr])


Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'model/tf_distil_bert_model_1/distilbert/embeddings/Gather_1' defined at (most recent call last):
    File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "/home/ifte-home/venvs/exp-Env/lib/python3.7/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/ifte-home/venvs/exp-Env/lib/python3.7/site-packages/traitlets/config/application.py", line 982, in launch_instance
      app.start()
    File "/home/ifte-home/venvs/exp-Env/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/ifte-home/venvs/exp-Env/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.7/asyncio/base_events.py", line

In [ ]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    print("Lowest Validation Loss: epoch {}".format(np.argmin(val_loss)+1))
    print("Highest Validation Accuracy: epoch {}".format(np.argmax(val_acc)+1))

plot_history(history)


In [ ]:
def get_min_val_loss_epoch(history):
    return "0"+str(np.argmin(history.history['val_loss'])+1)

def get_max_val_acc_epoch(history):
    return "0"+str(np.argmax(history.history['val_accuracy'])+1)

epoch_num = get_max_val_acc_epoch(history)
model.load_weights(output_dir+"/weights."+epoch_num+".hdf5") # Load in model weights


In [ ]:
y_test_probs = model.predict(x_test)

# Turn probabilities into an interger prediction
y_hat = []
for prob in y_test_probs:
    y_hat.append(np.argmax(prob))

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_hat))
print_cf1(y_test, y_hat)
print(classification_report(y_test, y_hat, target_names=labels_5))

In [ ]:
import pandas as pd
